# A First Shot at Deep Learning with PyTorch

In this notebook, we are going to take a baby step into the world of deep learning using PyTorch. There are a ton of notebooks out there that teach you the fundamentals of deep learning and PyTorch, so here the idea is to give you some basic introduction to deep learning and PyTorch at a very high level. Therefore, this notebook is targeting beginners but it can also serve as a review for more experienced developers.

After completion of this notebook, you are expected to know the basic components of training a basic neural network with PyTorch. I have also left a couple of exercises towards the end with the intention of encouraging more research and practise of your deep learning skills. 

---

**Author:** Elvis Saravia([Twitter](https://twitter.com/omarsar0) | [LinkedIn](https://www.linkedin.com/in/omarsar/))

**Complete Code Walkthrough:** [Blog post](https://medium.com/dair-ai/a-first-shot-at-deep-learning-with-pytorch-4a8252d30c75)

## Importing the libraries

Like with any other programming exercise, the first step is to import the necessary libraries. As we are going to be using Google Colab to program our neural network, we need to install and import the necessary PyTorch libraries.

In [ ]:
## The usual imports
import torch # 导入torch库
import torch.nn as nn # 导入torch库的神经网络库

## print out the pytorch version used
print(torch.__version__) # 打印一下torch的版本

## The Neural Network

![alt text](https://drive.google.com/uc?export=view&id=1Lpi4VPBfAV3JkOLopcsGK4L8dyxmPF1b)

Before building and training a neural network the first step is to process and prepare the data. In this notebook, we are going to use syntethic data (i.e., fake data) so we won't be using any real world data. 

For the sake of simplicity, we are going to use the following input and output pairs converted to tensors, which is how data is typically represented in the world of deep learning. The x values represent the input of dimension `(6,1)` and the y values represent the output of similar dimension. The example is taken from this [tutorial](https://github.com/lmoroney/dlaicourse/blob/master/Course%201%20-%20Part%202%20-%20Lesson%202%20-%20Notebook.ipynb). 

The objective of the neural network model that we are going to build and train is to automatically learn patterns that better characterize the relationship between the `x` and `y` values. Essentially, the model learns the relationship that exists between inputs and outputs which can then be used to predict the corresponding `y` value for any given input `x`.

In [ ]:
## our data in tensor form
x = torch.tensor([[-1.0],  [0.0], [1.0], [2.0], [3.0], [4.0]], dtype=torch.float) # 创建一个二维张量，大小是6行1列
y = torch.tensor([[-3.0], [-1.0], [1.0], [3.0], [5.0], [7.0]], dtype=torch.float) # 创建一个张量
# print(x, y)

In [ ]:
## print size of the input tensor
x.size() # 打印一下输入张量的大小

## The Neural Network Components
As said earlier, we are going to first define and build out the components of our neural network before training the model.

### Model

Typically, when building a neural network model, we define the layers and weights which form the basic components of the model. Below we show an example of how to define a hidden layer named `layer1` with size `(1, 1)`. For the purpose of this tutorial, we won't explicitly define the `weights` and allow the built-in functions provided by PyTorch to handle that part for us. By the way, the `nn.Linear(...)` function applies a linear transformation ($y = xA^T + b$) to the data that was provided as its input. We ignore the bias for now by setting `bias=False`.

In [ ]:
## Neural network with 1 hidden layer
""" nn.Linear
    args: 
        in_features (int) 输入样本的大小
        out_features (int) 输出样本的大小
        bias (bool) If set to False, the layer will not learn an additive bias. Default: True
"""
layer1 = nn.Linear(1, 1, bias=False) # 线性模型，b=0，输入大小是1，输出大小是1
model = nn.Sequential(layer1) # Sequential是一个顺序容器，可以存放多个模型，比如这里就放了一个线性回归模型（一层）

> 参考李沐的课程，他讲了手动实现一个module和sequential，方法是通过继承nn.module类。
> 
> 👇手动实现一个module
> 
> ![picture 1](../assets/images/1682582350241.png)
> ![picture 2](../assets/images/1682582421571.png)  
>
> 👇手动实现sequential
> 
> ![picture 3](../assets/images/1682582753803.png)  
>
> **通过上述对比，可以看出，手动实现一个module来完成预测操作，如果后续修改模型的话，需要修改整个module（因为这个module是写死某些层）。但是如果将一个大模型拆分，将各个层放入sequential中，后期想修改整体模型（如增加或删除一层）将变得很方便。**

### Loss and Optimizer
The loss function, `nn.MSELoss()`, is in charge of letting the model know how good it has learned the relationship between the input and output. The optimizer (in this case an `SGD`) primary role is to minimize or lower that loss value as it tunes its weights.

In [ ]:
## loss function
criterion = nn.MSELoss() # Mean Squared Error（均方误差）

## optimizer algorithm
optimizer = torch.optim.SGD(model.parameters(), lr=0.01) # 优化器是随机梯度下降，学习率是0.01

## Training the Neural Network Model
We have all the components we need to train our model. Below is the code used to train our model. 

In simple terms, we train the model by feeding it the input and output pairs for a couple of rounds (i.e., `epoch`). After a series of forward and backward steps, the model somewhat learns the relationship between x and y values. This is notable by the decrease in the computed `loss`. For a more detailed explanation of this code check out this [tutorial](https://medium.com/dair-ai/a-simple-neural-network-from-scratch-with-pytorch-and-google-colab-c7f3830618e0). 

In [ ]:
## training
for ITER in range(150):
    model = model.train() # model.train()的作用是启用 Batch Normalization 和 Dropout。来自https://zhuanlan.zhihu.com/p/357075502

    ## forward（下面这五步堪称经典）
    output = model(x) # 获得输入x的预测值
    loss = criterion(output, y) # 调用刚才的定义的损失函数
    optimizer.zero_grad() # 由于torch默认进行梯度累加，所以在反向传播计算梯度之前，先清零

    ## backward + update model params 
    loss.backward() # 反向传播计算梯度
    optimizer.step() # 更新参数

    # 训练完train样本后，生成的模型model要用来测试样本。在model(test)之前，需要加上model.eval()，否则的话，有输入数据，即使不训练，它也会改变权值。这是model中含有BN层和Dropout所带来的的性质。
    model.eval() # model.eval()的作用是不启用 Batch Normalization 和 Dropout。
    print('Epoch: %d | Loss: %.4f' %(ITER, loss.detach().item()))

## Testing the Model
After training the model we have the ability to test the model predictive capability by passing it an input. Below is a simple example of how you could achieve this with our model. The result we obtained aligns with the results obtained in this [notebook](https://github.com/lmoroney/dlaicourse/blob/master/Course%201%20-%20Part%202%20-%20Lesson%202%20-%20Notebook.ipynb), which inspired this entire tutorial. 

In [ ]:
## test the model
sample = torch.tensor([10.0], dtype=torch.float) # 测试样本
predicted = model(sample) # 预测结果
print(predicted.detach().item()) # 打印

## Final Words

Congratulations! In this tutorial you learned how to train a simple neural network using PyTorch. You also learned about the basic components that make up a neural network model such as the linear transformation layer, optimizer, and loss function. We then trained the model and tested its predictive capabilities. You are well on your way to become more knowledgeable about deep learning and PyTorch. I have provided a bunch of references below if you are interested in practising and learning more. 

*I would like to thank Laurence Moroney for his excellent [tutorial](https://github.com/lmoroney/dlaicourse/blob/master/Course%201%20-%20Part%202%20-%20Lesson%202%20-%20Notebook.ipynb) which I used as an inspiration for this tutorial.*

## Exercises
- Add more examples in the input and output tensors. In addition, try to change the dimensions of the data, say by adding an extra value in each array. What needs to be changed to successfully train the network with the new data?
- The model converged really fast, which means it learned the relationship between x and y values after a couple of iterations. Do you think it makes sense to continue training? How would you automate the process of stopping the training after the model loss doesn't subtantially change?
- In our example, we used a single hidden layer. Try to take a look at the PyTorch documentation to figure out what you need to do to get a model with more layers. What happens if you add more hidden layers?
- We did not discuss the learning rate (`lr-0.001`) and the optimizer in great detail. Check out the [PyTorch documentation](https://pytorch.org/docs/stable/optim.html) to learn more about what other optimizers you can use.

## References
- [The Hello World of Deep Learning with Neural Networks](https://github.com/lmoroney/dlaicourse/blob/master/Course%201%20-%20Part%202%20-%20Lesson%202%20-%20Notebook.ipynb)
- [A Simple Neural Network from Scratch with PyTorch and Google Colab](https://medium.com/dair-ai/a-simple-neural-network-from-scratch-with-pytorch-and-google-colab-c7f3830618e0?source=collection_category---4------1-----------------------)
- [PyTorch Official Docs](https://pytorch.org/docs/stable/nn.html)
- [PyTorch 1.2 Quickstart with Google Colab](https://medium.com/dair-ai/pytorch-1-2-quickstart-with-google-colab-6690a30c38d)
- [A Gentle Intoduction to PyTorch](https://medium.com/dair-ai/pytorch-1-2-introduction-guide-f6fa9bb7597c)